# Первым этапом для выполнения задания будет импорт необходимых питон-библиотек;

In [2]:
import pandas as pd
import pandahouse
import numpy as np
import scipy.stats as stats

# Далее выгрузим данные из кликхауса по датам из прошлого задания, но по всем экспериментальным группам;

In [3]:
connection_0 = {'host': 'https://clickhouse.lab.karpov.courses',
                      'database':'simulator_20221220',
                      'user':'student', 
                      'password':'dpo_python_2020'
}

query1 = """
SELECT exp_group, user_id, SUM(action = 'like') AS likes, SUM(action = 'view') AS views, likes/ views AS CTR
FROM {db}.feed_actions
WHERE (toDate(time) BETWEEN '2022-12-02' AND '2022-12-08' ) AND exp_group IN (0, 3)
GROUP BY exp_group, user_id
"""

query2 = """
SELECT exp_group, user_id, SUM(action = 'like') AS likes, SUM(action = 'view') AS views, likes/ views AS CTR
FROM {db}.feed_actions
WHERE (toDate(time) BETWEEN '2022-12-02' AND '2022-12-08' ) AND exp_group IN (1, 2)
GROUP BY exp_group, user_id
"""

df1, df2 = pandahouse.read_clickhouse(query1, connection = connection_0), pandahouse.read_clickhouse(query2, connection = connection_0)

# Посмотрим количество пользователей в каждой из групп;

In [4]:
df1.groupby('exp_group')['user_id'].count()

exp_group
0     9999
3    10049
Name: user_id, dtype: int64

In [5]:
df2.groupby('exp_group')['user_id'].count()

exp_group
1    10079
2     9952
Name: user_id, dtype: int64

# В группах примерно одинаковое количество пользователей; далее необхоимо посчитать общее значение CTR по контрольным группам 0 и 1, а далее посчитать линезированные лайки для всех групп;

In [6]:
control_CTR_0 = df1[df1.exp_group == 0].likes.sum() / df1[df1.exp_group == 0].views.sum()
control_CTR_1 = df2[df2.exp_group == 1].likes.sum() / df2[df2.exp_group == 1].views.sum()

print(control_CTR_0)
print(control_CTR_1)

0.20823612262916305
0.20802680490126244


In [7]:
df1['linerised_likes'] = df1.likes - control_CTR_0 * df1.views

In [8]:
df1

,exp_group,user_id,likes,views,CTR,linerised_likes
0,3,115383,9,30,0.300000,2.752916
1,3,123580,13,48,0.270833,3.004666
2,0,4944,8,41,0.195122,-0.537681
3,0,4504,5,15,0.333333,1.876458
4,0,121508,18,88,0.204545,-0.324779
...,...,...,...,...,...,...
20043,3,13781,10,27,0.370370,4.377625
20044,3,130651,18,98,0.183673,-2.407140
20045,0,3681,7,55,0.127273,-4.452987
20046,3,122008,5,41,0.121951,-3.537681


In [10]:
df2['linerised_likes'] = df2.likes - control_CTR_1 * df2.views

In [11]:
df2

,exp_group,user_id,likes,views,CTR,linerised_likes
0,1,109963,3,15,0.200000,-0.120402
1,1,26117,32,141,0.226950,2.668221
2,1,138232,18,73,0.246575,2.814043
3,1,26295,33,122,0.270492,7.620730
4,1,18392,7,32,0.218750,0.343142
...,...,...,...,...,...,...
20026,1,132567,45,296,0.152027,-16.575934
20027,1,30872,11,59,0.186441,-1.273581
20028,2,25383,42,127,0.330709,15.580596
20029,2,139020,24,114,0.210526,0.284944


# Теперь можно провести Т-тесты по обычному CTR и линезированным лайкам;

In [13]:
stats.ttest_ind(df1[df1.exp_group == 0].CTR, df1[df1.exp_group == 3].CTR, equal_var = False)

Ttest_indResult(statistic=-13.896870721904069, pvalue=1.055849414662529e-43)

In [14]:
stats.ttest_ind(df1[df1.exp_group == 0].linerised_likes, df1[df1.exp_group == 3].linerised_likes, equal_var = False)

Ttest_indResult(statistic=-15.21499546090383, pvalue=5.491424947968532e-52)

In [15]:
stats.ttest_ind(df2[df2.exp_group == 1].CTR, df2[df2.exp_group == 2].CTR, equal_var = False)

Ttest_indResult(statistic=0.7094392041270485, pvalue=0.4780623130874935)

In [16]:
stats.ttest_ind(df2[df2.exp_group == 1].linerised_likes, df2[df2.exp_group == 2].linerised_likes, equal_var = False)

Ttest_indResult(statistic=6.122579994775972, pvalue=9.439432187037712e-10)

# Т-тест по группа 0 и 3 изначально показал низкое Р-уровень значимости, а Т-тест на линезированных лайках еще сильнее уменьшил его значением;

# Т-тест, использовавшийся в задании 5.2 на группах 1 и 2, как и в прошлый раз, не показал статистически значимого результата, а на линезированных лайках Р-уровень значимости уменьшился достаточно до статистической значимости, соответственно, на обоих примерах этот метод оказался эффективным.